In [ ]:
import json
import ssl
import requests
from urllib.request import Request,urlopen
import pandas as pd
import numpy as np
import re
import string
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
from nltk.stem import PorterStemmer
from string import punctuation
from tqdm import tqdm
#import seaborn as sns
import spacy
import scipy

In [ ]:
import gensim

# Web Scrape Yelp Reviews to Analyze

In [ ]:
import bs4
from bs4 import BeautifulSoup



In [ ]:
#from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
#yelp='https://www.yelp.com/biz/fritto-misto-santa-monica-2'
yelp=requests.get('https://www.yelp.com/biz/fritto-misto-santa-monica-2?osq=fritto+misto')
#print(yelp)
html=BeautifulSoup(yelp.text,'html.parser')


In [ ]:
#url2 = 'https://www.yelp.com/biz/fritto-misto-santa-monica-2?osq=fritto+misto&start='
#start = 0
#num_pages = 10
#end = 20* num_pages

name=[]
reviews=[]
ratings=[]
date=[]
location=[]
urllist=[]

url_list=[]
for i in tqdm(range(0,3300,10)):
    URL = ('https://www.yelp.com/biz/fritto-misto-santa-monica-2?start='+str(i))
    yelp=requests.get(URL)
    html=BeautifulSoup(yelp.text,'html.parser')
#while (start<end):
    #url = url2+str(start)
    #print(url)
    #start +=10
    #print(url)
    rev=html.find_all('div',attrs={'class':'review__09f24__oHr9V border-color--default__09f24__NPAKY'})
    for review in rev:
        reviews.append(review.find('p').get_text())

    rati=html.find_all('div',{"class":"review__09f24__oHr9V border-color--default__09f24__NPAKY"})
    for i in rati:
        ratingshtml=i.find('span',{"class":"display--inline__09f24__c6N_k border-color--default__09f24__NPAKY"})
        star_rating=ratingshtml.div.get('aria-label')  
        bad_char=['star rating']
        for rating in bad_char:
            clean_rating=star_rating.replace(rating,"").strip()
        ratings.append(clean_rating)
        #print(ratings)
        
    dates=html.find_all('div',{"class":"review__09f24__oHr9V border-color--default__09f24__NPAKY"})
    for i in dates:
        datehtml=i.find('span',{"class":"css-chan6m"})
        date.append(datehtml.get_text())
        
    names=html.find_all('div',{"class":"review__09f24__oHr9V border-color--default__09f24__NPAKY"})
    for i in names:
        namehtml=i.find('span',{"class":"fs-block css-ux5mu6"})
        name.append(namehtml.get_text())
    
    loc=html.find_all('div',{"class":"review__09f24__oHr9V border-color--default__09f24__NPAKY"})
    for i in loc:
        try:
            locationhtml=i.find('span',{"class":"css-qgunke"})
            location.append(locationhtml.get_text())
        except AttributeError:
            location.append("No Location")
        #print(location)
    
    urltag=html.find_all('div',{"class":"review__09f24__oHr9V border-color--default__09f24__NPAKY"})
    for i in urltag:
        urlhtml=i.find('span',{"class":"fs-block css-ux5mu6"})
        for link in urlhtml.find_all('a'):
            urlname=link.get('href')
            urllist.append(urlname)
        
        
        #for link in urlhtml.find_all('a'):
        
            #links=[link.get('href') for link in urlhtml]
        #links=urlhtml.div.div.div.get('href')
        #print(links)
            #print(len(links))
            #url.append(links)#url.append(links)

In [ ]:
df=pd.read_csv('Fritto_Data.csv',index_col=0)

In [ ]:
df

In [ ]:
df.info()

In [ ]:
#Change to datetime
df['Date Posted'] = pd.to_datetime(df['Date Posted']).dt.date
df['Month Posted'] = df['Date Posted'].apply(lambda x: x.strftime('%m'))
df['Month_Year Posted'] = df['Date Posted'].apply(lambda x: x.strftime('%Y-%m'))
df['Year Posted']=df['Date Posted'].apply(lambda x: x.strftime('%Y'))
      

In [ ]:
df["Target"] = np.where(df["Ratings"] >= 4, 1, 0)


In [ ]:
df['Target'].value_counts()

In [ ]:
df=df.copy()

In [ ]:
df

In [ ]:
#df.to_csv('Fritto Data.csv')

In [ ]:
from urllib.parse import urljoin, quote_plus
standard_yelp_url='https://www.yelp.com'

new_url=[]
for i in df['Reviewer URL']:
    url_final=urljoin(standard_yelp_url,i)
    new_url.append(url_final)
    
df['Reviewer URL']=new_url

In [ ]:
#df.to_csv('Fritto_cust_output.csv')

from urllib.parse import urljoin, quote_plus
standard_yelp_url='https://www.yelp.com'
test=urljoin(standard_yelp_url,df['Reviewer URL'][1])

# NLP Reviews Exploratory Data Analysis

In [ ]:
#Natural Language Processing 

#Bad Reviews:
df_bad=df[df['Target']==0]
#Good Reviews:
df_good=df[df['Target']==1]



In [ ]:
#Wordcloud - Good reviews
from os import path
from wordcloud import WordCloud

text=""
for review in df_good['Reviews']:
    text=text+review
    
text=text.lower()
wordcloud=WordCloud().generate(text)

import matplotlib.pyplot as plt
plt.imshow(wordcloud)
plt.axis('off')

wordcloud=WordCloud(max_font_size=40).generate(text)
plt.figure()
plt.imshow(wordcloud)
plt.axis('off')
plt.show()

In [ ]:
#Wordcloud - Bad reviews
from os import path
from wordcloud import WordCloud

text=""
for review in df_bad['Reviews']:
    text=text+review
    
text=text.lower()
wordcloud=WordCloud().generate(text)

import matplotlib.pyplot as plt
plt.imshow(wordcloud)
plt.axis('off')

wordcloud=WordCloud(max_font_size=40).generate(text)
plt.figure()
plt.imshow(wordcloud)
plt.axis('off')
plt.show()

In [ ]:
#Bar Plot of Rating Breakdown
fig = plt.figure(figsize=(12,8))
axes1 = plt.subplot(2,2,1)
axes1 = sns.countplot(x='Ratings', data=df)
axes1.set_title('Ratings')
axes1.set_ylabel('Count')

In [ ]:
#Number of Reviews per Month
import matplotlib.pyplot as plt
df['Date Posted'] = pd.to_datetime(df['Date Posted'])
df = df.set_index('Date Posted')
plt.plot(df['Reviews'].resample('M').count())
plt.xlabel('Date')
plt.ylabel('Number of reviews')
plt.title('Number of reviews per month')
plt.show()

In [ ]:
# Countplot shows the distribution of monthly reviews - aggregate average
plt.rcParams['figure.figsize'] = [15, 8]
sns.set(font_scale = 1.2, style = 'whitegrid')
sns_year = sns.countplot(df['Month Posted'], color = 'darkcyan')
sns_year.set(xlabel = "Month", ylabel = "Count", title = "Distribution of the Reviews according to the Month")

#Summer months seem to be popular for reviews 

In [ ]:
#2022 Most popular months for reviews
df_2022=df[df['Year Posted']=='2022']
plt.rcParams['figure.figsize'] = [35, 18]
sns.set(font_scale = 1.2, style = 'whitegrid')
sns_year = sns.countplot(df_2022['Month Posted'], color = 'blue')
sns_year.set(xlabel = "Month", ylabel = "Count", title = "Distribution of the Reviews according to the Month")
plt.xticks(rotation = 90)

In [ ]:
#2021 Most popular months for reviews
df_2021=df[df['Year Posted']=='2021']
plt.rcParams['figure.figsize'] = [35, 18]
sns.set(font_scale = 1.2, style = 'whitegrid')
sns_year = sns.countplot(df_2021['Month Posted'], color = 'red')
sns_year.set(xlabel = "Month", ylabel = "Count", title = "Distribution of the Reviews according to the Month")
plt.xticks(rotation = 90)

In [ ]:
#2020 Most popular months for reviews
df_2020=df[df['Year Posted']=='2020']
plt.rcParams['figure.figsize'] = [35, 18]
sns.set(font_scale = 1.2, style = 'whitegrid')
sns_year = sns.countplot(df_2020['Month Posted'], color = 'green')
sns_year.set(xlabel = "Month", ylabel = "Count", title = "Distribution of the Reviews according to the Month")
plt.xticks(rotation = 90)

In [ ]:
#2021 Most popular months for reviews
df_2019=df[df['Year Posted']=='2019']
plt.rcParams['figure.figsize'] = [35, 18]
sns.set(font_scale = 1.2, style = 'whitegrid')
sns_year = sns.countplot(df_2019['Month Posted'], color = 'yellow')
sns_year.set(xlabel = "Month", ylabel = "Count", title = "Distribution of the Reviews according to the Month")
plt.xticks(rotation = 90)

In [ ]:
# Countplot shows the distribution of monthly reviews
plt.rcParams['figure.figsize'] = [35, 18]
sns.set(font_scale = 1.2, style = 'whitegrid')
sns_year = sns.countplot(df['Month_Year Posted'], color = 'darkcyan')
sns_year.set(xlabel = "Month", ylabel = "Count", title = "Distribution of the Reviews according to the Month")
plt.xticks(rotation = 90)
#Reviews seem to be increasing over time

In [ ]:
#Average Customer Review per Month - aggregate average
plt.plot(df['Ratings'].resample('M').mean())
plt.xlabel('Date')
plt.ylabel('Rating')
plt.title('Average Monthly Customer Rating')
plt.ylim(0,5)
plt.show()

In [ ]:
#2022 Average Monthly Customer Rating
plt.plot(df_2022['Ratings'].resample('M').mean())
plt.xlabel('Date')
plt.ylabel('Rating')
plt.title('Average Monthly Customer Rating')
plt.ylim(0,5)
plt.show()

In [ ]:
#2021 Average Monthly Customer Rating
plt.plot(df_2021['Ratings'].resample('M').mean())
plt.xlabel('Date')
plt.ylabel('Rating')
plt.title('Average Monthly Customer Rating')
plt.ylim(0,5)
plt.show()

In [ ]:
#2020 Average Monthly Customer Rating
plt.plot(df_2020['Ratings'].resample('M').mean())
plt.xlabel('Date')
plt.ylabel('Rating')
plt.title('Average Monthly Customer Rating')
plt.ylim(0,5)
plt.show()

In [ ]:
#2019 Average Monthly Customer Rating
plt.plot(df_2019['Ratings'].resample('M').mean())
plt.xlabel('Date')
plt.ylabel('Rating')
plt.title('Average Monthly Customer Rating')
plt.ylim(0,5)
plt.show()

# NLP Reviews Additional Analysis and Preprocessing

In [ ]:
df

In [ ]:
import nltk

from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import networkx as nx
import re
from nltk.stem import WordNetLemmatizer
import string

In [ ]:
lemma=WordNetLemmatizer()
stemmer=nltk.SnowballStemmer('english')
stop_words=set(stopwords.words('English'))

In [ ]:
stopwords.words('English')

In [ ]:
#Contractions List
contractions_dict = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"doesn’t": "does not",
"don't": "do not",
"don’t": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so is",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y’all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have",
"ain’t": "am not",
"aren’t": "are not",
"can’t": "cannot",
"can’t’ve": "cannot have",
"’cause": "because",
"could’ve": "could have",
"couldn’t": "could not",
"couldn’t’ve": "could not have",
"didn’t": "did not",
"doesn’t": "does not",
"don’t": "do not",
"don’t": "do not",
"hadn’t": "had not",
"hadn’t’ve": "had not have",
"hasn’t": "has not",
"haven’t": "have not",
"he’d": "he had",
"he’d’ve": "he would have",
"he’ll": "he will",
"he’ll’ve": "he will have",
"he’s": "he is",
"how’d": "how did",
"how’d’y": "how do you",
"how’ll": "how will",
"how’s": "how is",
"i’d": "i would",
"i’d’ve": "i would have",
"i’ll": "i will",
"i’ll’ve": "i will have",
"i’m": "i am",
"i’ve": "i have",
"isn’t": "is not",
"it’d": "it would",
"it’d’ve": "it would have",
"it’ll": "it will",
"it’ll’ve": "it will have",
"it’s": "it is",
"let’s": "let us",
"ma’am": "madam",
"mayn’t": "may not",
"might’ve": "might have",
"mightn’t": "might not",
"mightn’t’ve": "might not have",
"must’ve": "must have",
"mustn’t": "must not",
"mustn’t’ve": "must not have",
"needn’t": "need not",
"needn’t’ve": "need not have",
"o’clock": "of the clock",
"oughtn’t": "ought not",
"oughtn’t’ve": "ought not have",
"shan’t": "shall not",
"sha’n’t": "shall not",
"shan’t’ve": "shall not have",
"she’d": "she would",
"she’d’ve": "she would have",
"she’ll": "she will",
"she’ll’ve": "she will have",
"she’s": "she is",
"should’ve": "should have",
"shouldn’t": "should not",
"shouldn’t’ve": "should not have",
"so’ve": "so have",
"so’s": "so is",
"that’d": "that would",
"that’d’ve": "that would have",
"that’s": "that is",
"there’d": "there would",
"there’d’ve": "there would have",
"there’s": "there is",
"they’d": "they would",
"they’d’ve": "they would have",
"they’ll": "they will",
"they’ll’ve": "they will have",
"they’re": "they are",
"they’ve": "they have",
"to’ve": "to have",
"wasn’t": "was not",
"we’d": "we would",
"we’d’ve": "we would have",
"we’ll": "we will",
"we’ll’ve": "we will have",
"we’re": "we are",
"we’ve": "we have",
"weren’t": "were not",
"what’ll": "what will",
"what’ll’ve": "what will have",
"what’re": "what are",
"what’s": "what is",
"what’ve": "what have",
"when’s": "when is",
"when’ve": "when have",
"where’d": "where did",
"where’s": "where is",
"where’ve": "where have",
"who’ll": "who will",
"who’ll’ve": "who will have",
"who’s": "who is",
"who’ve": "who have",
"why’s": "why is",
"why’ve": "why have",
"will’ve": "will have",
"won’t": "will not",
"won’t’ve": "will not have",
"would’ve": "would have",
"wouldn’t": "would not",
"wouldn’t’ve": "would not have",
"y’all": "you all",
"y’all": "you all",
"y’all’d": "you all would",
"y’all’d’ve": "you all would have",
"y’all’re": "you all are",
"y’all’ve": "you all have",
"you’d": "you would",
"you’d’ve": "you would have",
"you’ll": "you will",
"you’ll’ve": "you will have",
"you’re": "you are",
"you’re": "you are",
"you’ve": "you have",
}
contractions_re = re.compile('(%s)' % '|'.join(contractions_dict.keys()))

def expand_contractions(s, contractions_dict=contractions_dict):
    def replace(match):
        return contractions_dict[match.group(0)]
    return contractions_re.sub(replace, s)

In [ ]:
#Cleaning the Tweets
def clean_tweet(tweet):
    tweet=str(tweet.lower())
    tweet = re.sub('https?:\/\/[a-zA-Z0-9@:%._\/+~#=?&;-]*', ' ', tweet)
    tweet = re.sub('\$[a-zA-Z0-9]*', ' ', tweet)
    tweet = re.sub('\@[a-zA-Z0-9]*', ' ', tweet)
    tweet = re.sub('[^a-zA-Z\']', ' ', tweet)
    tweet = expand_contractions(tweet)
    tweet = tweet.replace("'s", "")
    tweet = tweet.replace("’s", "")
    tweet = tweet.replace("\'s", "")
    tweet = tweet.replace("\’s", "")
    tweet=re.sub(' +', ' ',tweet)
    tweet = re.sub(' +', ' ',tweet)
    tweet = tweet.join(word for word in tweet if word not in punctuation)
    tweet = re.sub(' +', ' ',tweet)
    tweet = [word for word in tweet.split(' ') if word not in stop_words]
    tweet = " ".join(tweet)
    tweet = [stemmer.stem(word) for word in tweet.split(' ')]
    tweet =" ".join(tweet)
    tweet = tweet[1:]

    tweet=' '.join([lemma.lemmatize(x) for x in nltk.wordpunct_tokenize(tweet) if x not in stop_words])
    tweet=[lemma.lemmatize(x,nltk.corpus.reader.wordnet.VERB) for x in nltk.wordpunct_tokenize(tweet) if x not in stop_words]
    return tweet
        
    

In [ ]:
df['Clean Review']=df['Reviews'].apply(clean_tweet)   


In [ ]:
#df.to_csv('fritto_data_cleaned.csv')

In [ ]:
#df=pd.read_csv('fritto_data_cleaned.csv')

In [ ]:
from textblob import TextBlob
from collections import Counter

In [ ]:
def extract_keywords(tweet):
    tokens = nltk.word_tokenize(tweet)
    stop_words = nltk.corpus.stopwords.words('english')
    keywords = [token for token in tokens if token not in stop_words]
    lemmatizer = nltk.WordNetLemmatizer()
    keywords = [lemmatizer.lemmatize(keyword) for keyword in keywords]
    keyword_counts = Counter(keywords)
    return [keyword for keyword, count in keyword_counts.most_common(5)]

def analyze_sentiment(review):
    blob = TextBlob(review)
    sentiment_polarity = blob.sentiment.polarity
    if sentiment_polarity > 0:
        sentiment = 'positive'
    elif sentiment_polarity < 0:
        sentiment = 'negative'
    else:
        sentiment = 'neutral'
    return sentiment, sentiment_polarity

In [ ]:
#Extracting positive and negative keeywords
positive_keywords = []
negative_keywords = []
for review in df['Clean Review']:
    sentiment, polarity = analyze_sentiment(review)
    keywords = extract_keywords(review)
    if sentiment == 'positive':
        positive_keywords.extend(keywords)
    else:
        negative_keywords.extend(keywords)

In [ ]:
print('Most common positive keywords:', Counter(positive_keywords).most_common(5))
print('Most common negative keywords:', Counter(negative_keywords).most_common(5))

In [ ]:
from collections import Counter
from nltk.tokenize import word_tokenize

#Positive Review Keywords
positive_reviews_df = df[df["Target"] == 1]
positive_reviews = " ".join(positive_reviews_df["Clean Review"].tolist())
positive_reviews_tokens = word_tokenize(positive_reviews)
positive_reviews_keywords = Counter(positive_reviews_tokens).most_common(10)
print("Positive reviews keywords:")
print(positive_reviews_keywords)


In [ ]:
#Negative Review Keywords
negative_reviews_df = df[df["Target"] == 0]
negative_reviews = " ".join(negative_reviews_df["Clean Review"].tolist())
negative_reviews_tokens = word_tokenize(negative_reviews.lower())
negative_reviews_keywords = Counter(negative_reviews_tokens).most_common(10)
print("Negative review keywords:")
print(negative_reviews_keywords)


In [ ]:
from nltk import FreqDist
for review in positive_reviews_df['Clean Review']:
    for word in review:
        stemmed_token_dist = FreqDist(word)
        stemmed_dist = pd.DataFrame(stemmed_token_dist.most_common(20),columns=['Word', 'Frequency'])


In [ ]:
for review in negative_reviews_df['Clean Review']:
    for word in review:
        stemmed_token_dist = FreqDist(word)
        stemmed_dist = pd.DataFrame(stemmed_token_dist.most_common(20),columns=['Word', 'Frequency'])


In [ ]:
from nltk.tokenize import sent_tokenize

df_good=df[df['Ratings']==5]

good_reviews = ' '.join(df_good['Clean Review'])
# split the long string into sentences
sentences_good = sent_tokenize(good_reviews)
good_token_clean = list()
# get tokens for each sentence
for sentence in sentences_good:
    eng_word = re.findall(r'[A-Za-z\-]+', sentence)
    good_token_clean.append([i.lower() for i in eng_word if i.lower() not in stop_words])
    
good_token_clean

# Develop Sentiment Analysis Model

In [ ]:
import gensim
from gensim.models import Word2Vec
model_ted = Word2Vec(sentences=good_token_clean, size=500, window=10, min_count=1, workers=4, sg=0)
model_ted.predict_output_word(['service'], topn=10)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

x = df["Clean Review"]
y = df["Target"]



In [ ]:
import re
def words_and_char_bigrams(text):
    words = re.findall(r'\w{3,}', text)
    for w in words:
        yield w
        for i in range(len(w) - 2):
            yield w[i:i+2]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
bow_transformer = CountVectorizer(analyzer=words_and_char_bigrams).fit(x)


In [ ]:
len(bow_transformer.vocabulary_)

In [ ]:
x = bow_transformer.transform(x)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=4)

In [ ]:

#SMOTE the training data
sm = SMOTE(random_state=1)
X_bal, y_bal = sm.fit_resample(X_train, y_train)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(X_bal, y_bal)

In [ ]:
preds = nb.predict(X_test)


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
print(confusion_matrix(y_test, preds))
print('\n')
print(classification_report(y_test, preds))

# Test the model on Reviews

In [ ]:
#Positive Review Prediction
positive_review = df['Reviews'][59]
positive_review

In [ ]:
positive_review_transformed = bow_transformer.transform([positive_review])
nb.predict(positive_review_transformed)[0]


In [ ]:
#Negative Review Prediction
negative_review = df['Reviews'][309]
negative_review

In [ ]:
negative_review_transformed = bow_transformer.transform([negative_review])
nb.predict(negative_review_transformed)[0]

# Review Summarizer

In [ ]:
#Summarization algorithm comparison
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.summarizers.luhn import LuhnSummarizer
from sumy.summarizers.lex_rank import LexRankSummarizer

def summarize_review(review_text, max_sentences):
    parser = PlaintextParser.from_string(review_text, Tokenizer("english"))
    
    # Try using different summarization algorithms
    lsa_summarizer = LsaSummarizer()
    luhn_summarizer = LuhnSummarizer()
    lexrank_summarizer = LexRankSummarizer()

    lsa_summary = lsa_summarizer(parser.document, sentences_count=max_sentences)
    luhn_summary = luhn_summarizer(parser.document, sentences_count=max_sentences)
    lexrank_summary = lexrank_summarizer(parser.document, sentences_count=max_sentences)

    # Choose the best summary among different algorithms
    summaries = [lsa_summary, luhn_summary, lexrank_summary]
    best_summary = max(summaries, key=len)

    return " ".join([str(sentence) for sentence in best_summary])

# Example Yelp restaurant review
restaurant_review = df['Reviews'][1]

# Generate summary for the review with a maximum of 3 sentences
max_sentences = 3
summary = summarize_review(restaurant_review, max_sentences)
print("Original Review:\n", restaurant_review)
print("\nGenerated Summary (up to 3 sentences):\n", summary)


In [ ]:
#Aspect-based text summarization - can be used for next step: keyword review classification
import spacy

# Load English language model in spaCy
nlp = spacy.load('en_core_web_sm')

def aspect_based_summarization(review_text):
    # Process the review text using spaCy
    doc = nlp(review_text)

    # Initialize aspect-specific summaries
    aspect_summaries = {
        "food quality": [],
        "service": [],
        "ambiance": [],
        "prices": [],
        "overall": []
    }

    # Define aspect keywords
    aspect_keywords = {
        "food quality": ['quality','lemon','runny','lime','cuisine','takeout','vegan','gnocchi','vegetarian','fresh','well-prepared','stale','well','prepared','authentic','atomic','calamari','pillows','puffs','cheese','ricotta','dish','dessert','inedible','flourless','tiramisu','satisfying','caprese','turkey','steak','chicken','sausage','lasagna','jumbo','like','pasta','salt','bread','shrimp','chocolate','cake','cream','drinks','wine', "delicious",'taste', "tasty", "flavorful",'runny','flavorless','flavor',"fresh",'disgusting','penne','fettuccine','alfredo', "high-quality",'prepared', "well-prepared",'garlic','bland','unflavorful','oily','oil','greasy','cream','heavy','sauce','fried','filling','sauces','ravioli'],
        "service": ['host','listen','listening','delivery','call','wait','right away','service','slow','time','menu','hostess','table','seat','host','waiter','table','receptionist','quick','responsive','accommodating','waitress','seated','rude','took','order','ordering',"service",'min','minutes','server', "excellent",'check', "staff", "attentive","friendly", "helpful",'courteous', "prompt",'mean','answer', "efficient", "knowledgeable",'phone','polite'],
        "ambiance": ["ambiance",'clean','dirty','bathrooms','decor','atmosphere','temperature','AC','tables','toilet','kitchen','party','place','family','casual','formal','romantic','spot','festive','lively','intimate','loud','noisy','bar','seats','chairs','location', "atmosphere", "decor", "cozy", "inviting", "warm", "stylish", "elegant", "relaxed",'parking','building','seating','park','lights'],
        "prices": ["prices",'price','total','bill was','check was','worth','value','money','pricey','pricy','portions','portion size','quantity', "reasonable",'over-priced','over','priced','loaded', "affordable",'inexpensive','wallet', "cost", "expensive", "cheap", "wallet-friendly","large portions","large plate"],
        "overall": ['overall','fritto misto','santa monica','la','los angeles','card','somewhere different','somewhere else','course','refund','reviews','review',"best spot",'food','come again','avoid','meal','menu','back','return','choice','elsewhere', "experience", "visit",'would not return', "would return",'come back','place','italian','restaurant','website','app', "would recommend to a friend", "would return", "would not return",'people','crowd','italian','authentic','waste']
    }

    # Extract sentences related to each aspect
    for sentence in doc.sents:
        for aspect, keywords in aspect_keywords.items():
            for word in keywords:
                if word in sentence.text.lower():
                    aspect_summaries[aspect].append(sentence.text)
                    break

    # Generate summaries for each aspect
    for aspect, sentences in aspect_summaries.items():
        if sentences:
            aspect_summaries[aspect] = " ".join(sentences)
        else:
            aspect_summaries[aspect] = "No specific comments related to this aspect."

    return aspect_summaries

# Example Yelp restaurant review
restaurant_review = df['Clean Review'][1]

# Generate aspect-based summaries for the review
aspect_summaries = aspect_based_summarization(restaurant_review)

# Print aspect-based summaries
for aspect, summary in aspect_summaries.items():
    print(f"{aspect.capitalize()} Summary:\n{summary}\n")
